In [3]:
import datetime

import requests
from bs4 import BeautifulSoup as bs

---
### request+bs로 크롤링


---

### 시가총액 목록 사용하기

In [5]:
# 시가총액 300억 이하 목록 구하기
size_market_url_0 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=0&page="
size_market_url_1 = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok=1&page="
codeL = []

# 코스피 300억 이하 (후보)
for i in range(19, 33):
    soup = bs(requests.get(size_market_url_0+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL

# 코스닥 300억 이하 (후보)
for i in range(22, 30):
    soup = bs(requests.get(size_market_url_1+str(i)).text, 'html.parser')
    tempL = [x.attrs["href"].split("code=")[1] for x in soup.select("table ")[1].select_one("tbody").select("a.tltle")]
    codeL += tempL
    
# 시가총액 작은 애들부터 보기
codeL.reverse()


In [7]:
# 오늘의 시가총액 code 목록 저장 
todayDate = datetime.datetime.now().strftime("%Y_%b_%d")
f = open("./"+todayDate+"_시가총액_하위권.txt", 'w')
f.write("\n".join(codeL))
f.close()

# 오늘의 시가총액 code 목록 불러오기
useDate = todayDate
# useDate = "2020_Feb_18"
f = open("./"+useDate+"_시가총액_하위권.txt", "r")
codeL = [x.replace("\n", "") for x in f.readlines()]
f.close()

---

<b>base code</b>


In [9]:
def getSoupFromCode(code):
    code_url = "https://finance.naver.com/item/main.nhn?code="
    soup = bs(requests.get(code_url + code).text, 'html.parser')
    return soup

def cleanText(text):
    return text.replace("\n", "").replace("\t","").replace("조","").replace(",","")

def getName(soup):
    name = soup.select_one("#middle > div.h_company > div.wrap_company > h2 > a").text
    return name

def getSoupFromReport(code):
    report_url = "https://navercomp.wisereport.co.kr/v2/company/c1050001.aspx?cn=&cmp_cd="
    soup = bs(requests.get(report_url + code).text, 'html.parser')
    return soup

def getJsonFromConsen(code):
    consen_url = "https://navercomp.wisereport.co.kr/company/ajax/c1050001_data.aspx?flag=2&finGubun=MAIN&frq=0"+\
        "&chartType=svg&cmp_cd="
    return requests.get(consen_url + code).json()

# def increasing(values):
    

---

## 사용할 조건들

- PBR 1 미만
- PER 7 미만
- 시총 300 미만
- PCR 8 미만
- POR 8 미만


---

#### etf 인가

In [10]:
def isETF(soup):
    if(len(soup.select(".e_summary")) == 0):
        return False
    return True

#### etn 인가

In [11]:
def isETN(soup):
    name = getName(soup)
    if("ETN" in name):
        return True
    return False

---

#### 우선주인가


In [12]:
def isFirst(soup):
    name = getName(soup)
    if(name[-1] == "우"):
        return True
    if(name[-2:] in ["우A", "우B", "우C"]):
        return True
    return False

#### 시총 300억 이하인가

In [21]:
def under300(soup):
    if(int(cleanText(soup.select_one("#_market_sum").text)) <= 300):
        return True
    return False    

#### per 7 미만인가

In [15]:
def perUnder7(soup_report):
    try:
        per_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PER"][0]
        per_float = float(cleanText(per_str))
        if(per_float < 7 and per_float > 0):
            return per_float
        return -1
    except:
        return -1
    

#### pbr 1 미만인가

In [16]:
def pbrUnder1(soup_report):
    try:
        pbr_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "PBR"][0]
        pbr_float = float(cleanText(pbr_str))
        if(pbr_float < 1 and pbr_float > 0):
            return pbr_float
        return -1
    except:
        return -1


---

#### 산업평균 per이 양수인가

In [19]:
def per_avg(soup_report):
    try:
        per_str = [x.text.split()[1] for x in soup_report.select_one(".td0301").select(".line-left") if x.text.split()[0] == "업종PER"][0]
        per_float = float(cleanText(per_str))
        if(per_float > 0):
            return per_float
        return -1
    except:
        return -1

In [14]:
json = getJsonFromConsen("900080")

In [15]:
salesL = [float(cleanText(x["SALES"])) for x in json["JsonData"] if x["SALES"] != ""]

In [16]:
# 영업이익
opL = [float(cleanText(x["OP"])) for x in json["JsonData"] if x["SALES"] != ""]
perL = [float(cleanText(x["PER"])) for x in json["JsonData"] if x["SALES"] != ""]
roeL = [float(cleanText(x["ROE"])) for x in json["JsonData"] if x["SALES"] != ""]

In [17]:
opL

[435.3, 324.4, 294.8, 219.1]

In [190]:
perL

[5.33, 4.76, 5.41, 3.74]

In [191]:
roeL

[8.42, 7.24, 4.96, 4.13]

In [150]:
soup_report.select_one("#cTB511")

In [134]:
soup_report.select("#cTB511")

[]

---

## 괜찮은 종목 거르기

In [22]:
good = []
for code in codeL:
    
    # 자원 확보
    soup = getSoupFromCode(code)
    soup_report = getSoupFromReport(code)
    name = getName(soup)
    
    # etf/etn/우선주/300초과 제외
    if(isETF(soup) or isETN(soup) or isFirst(soup) or under300(soup)):
        continue
        
    # 0<per<7, 0<pbr<1
    per = perUnder7(soup_report)
    pbr = pbrUnder1(soup_report)
    perAvg = per_avg(soup_report)
    if(per == -1 or pbr
       == -1 or perAvg == -1):
        continue
        
    # 출력
    good.append(name + " " + str(per) + " " + str(pbr))
    print(name, per, pbr)

원일특강 3.18 0.27
아즈텍WB 6.83 0.34
코리아에셋투자증권 4.52 0.52
한국컴퓨터 4.66 0.36
유비벨록스 6.32 0.47
바이오로그디바이스 6.54 0.73
오리콤 6.76 0.48
에코바이오 5.84 0.67
파워넷 6.99 0.7
한솔로지스틱스 5.7 0.72
SJM 5.26 0.22
동원수산 6.29 0.75


In [23]:
len(good)

12

In [24]:
good

['원일특강 3.18 0.27',
 '아즈텍WB 6.83 0.34',
 '코리아에셋투자증권 4.52 0.52',
 '한국컴퓨터 4.66 0.36',
 '유비벨록스 6.32 0.47',
 '바이오로그디바이스 6.54 0.73',
 '오리콤 6.76 0.48',
 '에코바이오 5.84 0.67',
 '파워넷 6.99 0.7',
 '한솔로지스틱스 5.7 0.72',
 'SJM 5.26 0.22',
 '동원수산 6.29 0.75']

---

## 나의 네이버증권 셋팅하기

In [35]:
!sudo pip3 install selenium -dhtlqwh50

Password:


In [39]:
from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

import time

ModuleNotFoundError: No module named 'selenium'

In [217]:
# 미리 로그인
driver = webdriver.Chrome('./chromedriver')
itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL"
driver.get(itemListUrl)

NameError: name 'webdriver' is not defined

In [16]:
for group in range(6, 9):

    # 링크 접속 및 로그인
    itemListUrl = "https://finance.naver.com/mystock/itemList.nhn?groupId=%s&type=OVERALL" % str(group)
    print(itemListUrl )
    
    # 기존 관심 종목 삭제
    time.sleep(0.5)
    driver.get(itemListUrl)
    time.sleep(0.5)
    driver.find_element_by_css_selector("#all_select").click()
    time.sleep(0.5)
    driver.find_element_by_css_selector("#wrap > div.section_mys > div.group_mystb.NE\=a\:lst > div:nth-child(2) > button.btn_del.NPI\=a\:del").click()
    time.sleep(0.5)
    Alert(driver).accept()
    
    # 적합 종목 리스트 찾기
    if(group == 6):
        useL = score25_code.copy()
    elif(group == 7):
        useL = score23_code.copy()
    elif(group == 8):
        useL = score20_code.copy()
            
    # 관심종목 목록 등록
    for nowCode in useL:
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").clear()
        time.sleep(0.5)
        driver.find_element_by_css_selector("#mystock_input").send_keys(nowCode)
        driver.find_element_by_css_selector("#mystock_input").send_keys(Keys.RETURN)


https://finance.naver.com/mystock/itemList.nhn?groupId=6&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=7&type=OVERALL
https://finance.naver.com/mystock/itemList.nhn?groupId=8&type=OVERALL
